#  RemoteCanClient

> RemoteCanClient class

In [ ]:
#| default_exp conn.remote_can_client

In [ ]:
#| export
import json
from threading import BoundedSemaphore
from typing import Dict, Optional
import logging

In [ ]:
#| export
import pandas as pd
import requests
import urllib3
from requests.adapters import HTTPAdapter

In [ ]:
#| export
from data_io_nbdev.config.vehicles import Truck, trucks_by_id
from data_io_nbdev.conn.remotecan.exceptions import RemoteCanException  # type: ignore

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#| export
DEFAULT_TIMEOUT = 5  # seconds

In [ ]:
#| export
class TimeoutHTTPAdapter(HTTPAdapter):
    """
    TimeoutHTTPAdapter is used to set a default timeout for all requests


    Reference: https://findwork.dev/blog/advanced-usage-python-requests-timeouts-retries-hooks/#retry-on-failure
    """

    def __init__(self, *args, **kwargs):
        self.timeout = DEFAULT_TIMEOUT
        if "timeout" in kwargs:
            self.timeout = kwargs["timeout"]
            del kwargs["timeout"]
        super().__init__(*args, **kwargs)

    def send(self, request, **kwargs):
        timeout = kwargs.get("timeout")
        if timeout is None:
            kwargs["timeout"] = self.timeout
        return super().send(request, **kwargs)

In [ ]:
#| export
class RemoteCanClient:
    """
    RemoteCanClient is used to send torque map to the remote can server and get observation signals from the server.

    Attributes:

        url: str
            The url of the remote can server, generated from host and port.
        host: str
            The host of the remote can server.
        port: str
            The port of the remote can server.
        truck: Truck
            The truck object.
        proxies: Optional[Dict]
            The proxies used to connect to the remote can server.
        max_connections: int
            The maximum number of connections to the remote can server.
        logger: Optional[logging.Logger]
            The logger used to log the information.
        dict_logger: Optional[Dict]
            The dictionary logger used to log the information.
    """

    def __init__(
        self,
        host: str,  # host name for remote can server
        port: str,  # port number for remote can server
        truck: Truck,  # truck object which RemoteCanClient is attached to
        proxies: Optional[Dict] = None,  # optional proxies for remote can server
        max_connections: int = 1,  # maximum number of connections to remote can server
        logger: Optional[logging.Logger] = None,  # logger for RemoteCanClient
        dict_logger: Optional[Dict] = None,  # dictionary logger for RemoteCanClient
    ):
        # url: str = "http://10.0.64.78:5000/",
        self.logger = logger.getChild("remote_can_client")
        self.dict_logger = dict_logger
        self.url = "http://" + host + ":" + port + "/"  # type: ignore
        self.proxies = proxies
        self.logger.info(
            f"{{'header': 'remote can url: {self.url}, proxy: {proxies}'}}",
            extra=self.dict_logger,
        )
        self.session = requests.Session()
        self.connect_sema = BoundedSemaphore(
            max_connections
        )  # the semaphore value depends on the server capability, current only 1 is allowed

        # Retry Policy
        total = 3
        connect = 5
        backoff_factor = 0.1
        status_forcelist = [413, 429, 500, 502, 503, 504]
        methods_allowed = [
            "HEAD",
            "GET",
            "PUT",
            "DELETE",
            "OPTIONS",
            "TRACE",
            "POST",
        ]

        self.retries = urllib3.util.Retry(
            total=total,
            connect=connect,
            backoff_factor=backoff_factor,
            status_forcelist=status_forcelist,
            allowed_methods=methods_allowed,
        )
        self.logger.info(
            f"{{'header': {{'retries': {total}, "
            f"'connect': {connect}, "
            f"'backoff_factor': {backoff_factor}, "
            f"'status_forcelist': {status_forcelist}, "
            f"'method_allowed': {methods_allowed}}}",
            extra=self.dict_logger,
        )
        # self.session.mount("http://", requests.adapters.HTTPAdapter(max_retries=retries))
        # self.session.mount("http://", TimeoutHTTPAdapter(max_retries=retries))
        # self.session.mount("http://", a)
        if proxies is not None:
            self.session.proxies.update(proxies)

        # self.vin = self.truck.vin
        # self.PedalScale = self.truck.pedal_scale  # 17
        # self.PedalRange = self.truck.PedalRange  # [0, 1.0]
        # self.VelocityScale = self.truck.VelocityScale  # 14
        # self.VelocityRange = self.truck.VelocityRange  # [0, 120]
        # self.PedalScaleList = self.truck.PedalScaleList
        # self.VelocityScaleList = self.truck.VelocityScaleList

        self.truck = truck
        self.logger.info(
            f"vehicle name: {self.truck.vid}, VIN: {self.truck.vin}",
            extra=self.dict_logger,
        )

    def send_torque_map(
        self,
        pedal_map: pd.DataFrame,  # The torque map to be sent to the remote can server.
        swap: bool = True,  # Whether to swap the pedal map.
        timeout=32,  # The timeout for the request.
    ) -> Dict:  # The response from the remote can server if successful
        """
        Send torque map to the remote can server.

        Args:
            pedal_map: pd.DataFrame
                The torque map to be sent to the remote can server.
            swap: bool
                Whether to swap the pedal map.
            timeout: int
                The timeout for the request.

        Returns:
            json_ret: Dict
                The response from the remote can server if successful.
        """

        self.logger.info("entering send_torque_map", extra=self.dict_logger)
        assert not any(
            pedal_map.index.duplicated()
        ), f"Input velocity index has duplicated values!"
        assert not any(
            pedal_map.columns.duplicated()
        ), f"Input pedal index has duplicated values!"
        assert set(pedal_map.columns.astype(float)) == (
            set(self.truck.pedal_scale)
        ), f"Unregurlar input map thrust index {pedal_map.columns} for {self.truck.vid}!"
        # assert set(pedalmap.index).issubset(set(self.VelocityScaleList)), \
        #     f"input map velocity index {pedalmap.index} is out of range for {self.truck.vid}!"
        input_row_num = len(pedal_map.index)
        assert any(
            self.truck.speed_scale[idx : idx + input_row_num] == list(pedal_map.index)
            for idx in range(len(self.truck.speed_scale) - input_row_num + 1)
        ), f"Input velocity index is not continuous!"

        self.logger.info("done assert!", extra=self.dict_logger)

        k = pedal_map.index[0]  # starting line
        start_row = self.truck.speed_scale.index(k)

        json_data = {
            "torque": pedal_map.values.tolist(),
            "vin": self.truck.vin,
            "abswitch": swap,
            "start_line": start_row,
        }

        try:
            self.session.mount(
                "http://", TimeoutHTTPAdapter(max_retries=self.retries, timeout=timeout)
            )
            with self.connect_sema:  # max sure the http connection is available (default capacity is 1)
                response = self.session.post(
                    self.url + "set_torque", json=json_data, timeout=timeout
                )
        except requests.exceptions.ConnectionError as e:
            raise RemoteCanException(err_code=1000, extra_msg=None, codes=None) from e
        except (requests.exceptions.ConnectTimeout, requests.exceptions.Timeout) as e:
            raise RemoteCanException(err_code=1002, extra_msg=None, codes=None) from e
            # return RetCode.network_timeout, {"request exception": "Timeout"}
        except Exception as e:
            raise RemoteCanException(err_code=1001, extra_msg=None, codes=None) from e
            # return RetCode.network_unknown_error, {"request exception": e}

        if response.status_code == 200:
            json_ret = json.loads(response.text)

            if json_ret["success"]:
                return {"json_ret": json_ret}
            elif "AIMode" in json_ret and not json_ret["AIMode"]:
                raise RemoteCanException(err_code=2, extra_msg=None, codes=None)
            else:
                try:
                    raise RemoteCanException(
                        err_code=int(json_ret["reason"]["why"]["code"]),
                        codes=None,
                        extra_msg=f"{{'json_ret': '{json_ret}'}}",
                    )
                    # ret_code = RetCode().map_string_code(string_code)
                    # return ret_code, {"json_ret": json_ret}
                except KeyError as e:
                    raise RemoteCanException(
                        err_code=2005,
                        codes=None,
                        extra_msg=f"{{'json_ret': '{json_ret}'}}",
                    ) from e
                    # return RetCode.remote_can_unknown_format, {"json_ret": json_ret}
        else:
            raise RemoteCanException(
                err_code=1,
                codes=None,
                extra_msg=f"{{'status_code': '{response.status_code}', "
                f"'text': '{json.loads(response.text)}'}}",
            )
            # self.logger.info(
            #     f"{{\'header\': \'Flash error\', "
            #     f"\'ret_code\': {RetCode.client_cannot_connect_to_server}, "
            #     f"\'requests_code\': {response.status_code}}}",
            #     extra=self.dict_logger,
            # )
            # return RetCode.client_cannot_connect_to_server, {
            #     "status code": response.text
            # }

    def get_signals(
        self,
        duration,  # The duration of the signals to be retrieved.
        timeout=32,  # The timeout for the request.
    ) -> (
        Dict
    ):  # The response from the remote can server (a dictionary containing the observation) if successful
        """
        Get observation signals from the remote can server.

        Args:
            duration: int
                The duration of the signals to be retrieved.
            timeout: int
                The timeout for the request.

        Return:
            json_ret: Dict
                The response from the remote can server (a dictionary containing the observation) if successful.
        """

        self.logger.info("entering get_signals!!!", extra=self.dict_logger)
        json_data = {"vin": self.truck.vin, "signal_duration": duration}

        try:
            self.session.mount(
                "http://", TimeoutHTTPAdapter(max_retries=self.retries, timeout=timeout)
            )
            with self.connect_sema:  # max sure the http connection is available (default capacity is 1)
                response = self.session.post(
                    self.url + "get_signal",
                    json=json_data,
                    timeout=timeout,
                )
            elapsed = response.elapsed.total_seconds()
            self.logger.info(
                f"{{'header': 'get_signal response', "
                f"'elapsed': {elapsed}"
                f"'timeout': {timeout}! "
                f"'tail': 'seconds'}}",
                extra=self.dict_logger,
            )
        except requests.exceptions.ConnectionError as e:
            raise RemoteCanException(err_code=1000) from e
            # self.logger.warning(
            #     f"{{\'ret_code\': {RetCode.network_connection_error}}}",
            #     extra=self.dict_logger,
            # )
            # return RetCode.network_connection_error, {"request exception": e}
        except (requests.exceptions.Timeout, requests.exceptions.ConnectTimeout) as e:
            raise RemoteCanException(err_code=1002) from e
            # self.logger.warning(
            #     f"{{\'ret_code\': {RetCode.network_timeout}}}", extra=self.dict_logger
            # )
            # return RetCode.network_timeout, {"request exception": e}
        except Exception as e:
            raise RemoteCanException(err_code=1001) from e
            # self.logger.warning(
            #     f"{{\'ret_code\': {RetCode.network_unknown_error}, "
            #     f"\'exception\': \'{e}\'}}",
            #     extra=self.dict_logger,
            # )
            # return RetCode.network_unknown_error, {"request exception": e}

        if response.status_code == 200:
            json_ret = json.loads(response.text)
            if json_ret["success"]:
                self.logger.info(
                    f"{{'header': 'get_signal Success!'}}", extra=self.dict_logger
                )
                return json_ret
            elif "AIMode" in json_ret and not json_ret["AIMode"]:
                raise RemoteCanException(
                    err_code=2, extra_msg=f"{{'json_ret': '{json_ret}'}}"
                )
                # self.logger.warning(
                #     f"{{\'header\': \'Blocked by AI mode switch!\'}}",
                #     extra=self.dict_logger,
                # )
                # return RetCode.ai_mode_shutdown, json_ret
            else:
                try:
                    raise RemoteCanException(
                        err_code=int(json_ret["reason"]["why"]["code"]),
                        extra_msg=f"{{'json_ret': '{json_ret}'}}",
                    )
                except KeyError as e:
                    raise RemoteCanException(
                        err_code=2005, extra_msg=f"{{'json_ret': '{json_ret}'}}"
                    ) from e
                # string_code = json_ret["reason"]["why"]["code"]
                # ret_code = RetCode().map_string_code(string_code)
                # self.logger.warning(
                #     f"{{\'ret_code\': {ret_code}}}", extra=self.dict_logger
                # )
                # return ret_code, json_ret
        else:
            raise RemoteCanException(
                err_code=1,
                extra_msg=f"{{'status_code': '{response.status_code}', "
                f"'text': '{json.loads(response.text)}'}}",
            )
            # self.logger.warning(
            #     f"{{\'header\': \'Server Failure\', "
            #     f"\'requests_code\': {response.status_code}}}",
            #     extra=self.dict_logger,
            # )
            # json_ret = json.loads(response.text)
            #
            # return RetCode.client_cannot_connect_to_server, json_ret

    def close(self):
        self.session.close()

In [ ]:
show_doc(RemoteCanClient.send_torque_map)

In [ ]:
show_doc(RemoteCanClient.get_signals)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()